# Importing Libraries


In [680]:
from helper import *

In [679]:
import importlib
importlib.reload(helper)


<module 'helper' from 'c:\\Users\\satya\\OneDrive\\Desktop\\Summer_Internship\\helper.py'>

# Data Collection and cleaning

In [353]:
# basic variables

start_date = '2000-01-01'
end_date = '2023-12-31' 
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [354]:
symbols = collect_tickers('US', start_date)

Sucesfully downloaded 835 symbols.


In [355]:
snp_stocks_data = download_data(symbols, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2000-01-01 -> 2024-06-04)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Could not fetch data for nan: 'float' object has no attribute 'replace'
Could not fetch data for nan: 'float' object has no attribute 'replace'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ATVI']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed


...

$DJ: possibly delisted; No price data found  (1d 2000-01-01 -> 2024-06-04)
$AV: possibly delisted; No price data found  (1d 2000-01-01 -> 2024-06-04)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['NCR']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MXIM']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FDC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed


Could not fetch data for nan: 'float' object has no attribute 'replace'


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TSG']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['QTRN']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2000-01-01 -> 2024-06-04)')


$QTRN: possibly delisted; No price data found  (1d 2000-01-01 -> 2024-06-04)
Sucesfully downloaded data for 799 stocks!


In [356]:
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)

Could not update data for 16 stocks.
Could not update data for 3 stocks.


In [357]:
symbols = list(snp_stocks_data.keys())

In [358]:
returns_df = calculate_return(symbols, snp_stocks_data)

In [363]:
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_momentum(data)

# Convert the momentum data dictionary to a DataFrame
momentum = pd.DataFrame.from_dict(momentum_data, orient='index')

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\1161350933.py:9: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\1161350933.py:10: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [364]:
# Calculate the 12-1 momentum return for each stock in the snp_stocks_data dictionary
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_12_1_momentum(data)

# Convert the momentum data dictionary to a DataFrame
momentum_12_1 = pd.DataFrame.from_dict(momentum_data, orient='index')

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:189: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:190: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [555]:
top_p_momentum_stocks_dict = top_p_percentile_stocks(momentum, 10)

In [379]:
top_p_momentum_stocks_dict_12_1 = top_p_percentile_stocks(momentum_12_1, 10)

In [380]:
bottom_p_momentum_stocks_dict = bottom_p_percentile_stocks(momentum, 10)

In [381]:
bottom_p_momentum_stocks_dict_12_1 = bottom_p_percentile_stocks(momentum_12_1, 10)

In [556]:
long_dict = total_return(snp_stocks_data, top_p_momentum_stocks_dict, returns_df)

In [557]:
long_dict

{Timestamp('2001-01-31 00:00:00'): 0,
 Timestamp('2001-02-28 00:00:00'): 0,
 Timestamp('2001-03-31 00:00:00'): 0.054624383945983054,
 Timestamp('2001-04-30 00:00:00'): 0.024146670892420116,
 Timestamp('2001-05-31 00:00:00'): 0,
 Timestamp('2001-06-30 00:00:00'): 0.02162487307787101,
 Timestamp('2001-07-31 00:00:00'): 0,
 Timestamp('2001-08-31 00:00:00'): 0,
 Timestamp('2001-09-30 00:00:00'): 0.03714608803272795,
 Timestamp('2001-10-31 00:00:00'): 0.07825469679754792,
 Timestamp('2001-11-30 00:00:00'): 0.07274338208460712,
 Timestamp('2001-12-31 00:00:00'): 0.020627473934484743,
 Timestamp('2002-01-31 00:00:00'): 0.053328180833021016,
 Timestamp('2002-02-28 00:00:00'): 0.04517858880268532,
 Timestamp('2002-03-31 00:00:00'): 0.04056254831114115,
 Timestamp('2002-04-30 00:00:00'): 0,
 Timestamp('2002-05-31 00:00:00'): 0,
 Timestamp('2002-06-30 00:00:00'): 0,
 Timestamp('2002-07-31 00:00:00'): 0.008940759925310779,
 Timestamp('2002-08-31 00:00:00'): 0,
 Timestamp('2002-09-30 00:00:00'): 0.

In [450]:
long_dict_12_1 = total_return(snp_stocks_data, top_p_momentum_stocks_dict_12_1, returns_df)

2001-01-31 00:00:00 3460.279667854309 -237.43850428234185
2001-02-28 00:00:00 1506.63846886158 -58.04871064600965
2001-03-31 00:00:00 1424.244421362877 95.23760161450397
2001-04-30 00:00:00 1346.0042473077774 50.027797609836895
2001-05-31 00:00:00 1411.8247665166855 -19.764520761193225
2001-06-30 00:00:00 1397.8078215122223 9.142340938431706
2001-07-31 00:00:00 1447.001630306244 -38.13032194184227
2001-08-31 00:00:00 1574.6510210037231 -104.44501143362687
2001-09-30 00:00:00 1336.0239235758781 73.9459093868143
2001-10-31 00:00:00 1365.1899669617414 105.5402272010728
2001-11-30 00:00:00 1329.7014397382736 85.9347499187797
2001-12-31 00:00:00 1556.4078124314547 46.069436329309646
2002-01-31 00:00:00 1640.4702393561602 89.06980808821571
2002-02-28 00:00:00 1645.1114134788513 100.22012567325112
2002-03-31 00:00:00 1718.6765937805176 62.661533868506346
2002-04-30 00:00:00 1839.5599611252546 -29.16074314779128
2002-05-31 00:00:00 1908.7626209706068 -87.89289412434997
2002-06-30 00:00:00 1942

In [451]:
long_dict_inversion = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

2001-01-31 00:00:00 1188.9879549257457 -145.5523993170941
2001-02-28 00:00:00 1598.2949832230806 -209.4592029721657
2001-03-31 00:00:00 1610.169915765524 332.87896613195767
2001-04-30 00:00:00 1172.8978600986302 8.327278708213688
2001-05-31 00:00:00 1303.067796330899 -33.18409756081352
2001-06-30 00:00:00 1702.9584515094757 -99.63281537847165
2001-07-31 00:00:00 2427.280877634883 -399.2779959899274
2001-08-31 00:00:00 2285.8567654937506 -451.20133340528673
2001-09-30 00:00:00 1653.793671131134 358.1929880992194
2001-10-31 00:00:00 1493.605034649372 132.57465724047998
2001-11-30 00:00:00 1805.4973699450493 65.51395515286333
2001-12-31 00:00:00 1922.4536390602589 -136.28910427018306
2002-01-31 00:00:00 2182.0331618487835 -59.67299515093406
2002-02-28 00:00:00 1890.758851736784 240.4510402244196
2002-03-31 00:00:00 2750.662007704377 -88.15749894487445
2002-04-30 00:00:00 2276.0930092930794 -78.01453354555606
2002-05-31 00:00:00 1252.1461555361748 -153.5440618880024
2002-06-30 00:00:00 133

In [452]:
sort_dict = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

2001-01-31 00:00:00 1188.9879549257457 -145.5523993170941
2001-02-28 00:00:00 1598.2949832230806 -209.4592029721657
2001-03-31 00:00:00 1610.169915765524 332.87896613195767
2001-04-30 00:00:00 1172.8978600986302 8.327278708213688
2001-05-31 00:00:00 1303.067796330899 -33.18409756081352
2001-06-30 00:00:00 1702.9584515094757 -99.63281537847165
2001-07-31 00:00:00 2427.280877634883 -399.2779959899274
2001-08-31 00:00:00 2285.8567654937506 -451.20133340528673
2001-09-30 00:00:00 1653.793671131134 358.1929880992194
2001-10-31 00:00:00 1493.605034649372 132.57465724047998
2001-11-30 00:00:00 1805.4973699450493 65.51395515286333
2001-12-31 00:00:00 1922.4536390602589 -136.28910427018306
2002-01-31 00:00:00 2182.0331618487835 -59.67299515093406
2002-02-28 00:00:00 1890.758851736784 240.4510402244196


2002-03-31 00:00:00 2750.662007704377 -88.15749894487445
2002-04-30 00:00:00 2276.0930092930794 -78.01453354555606
2002-05-31 00:00:00 1252.1461555361748 -153.5440618880024
2002-06-30 00:00:00 1338.8085570037365 -98.19049470770538
2002-07-31 00:00:00 1129.4119975566864 28.41547992766715
2002-08-31 00:00:00 778.731990814209 -109.40295591053211
2002-09-30 00:00:00 763.8915457427502 75.93751985575803
2002-10-31 00:00:00 548.1270298659801 100.25123658317075
2002-11-30 00:00:00 886.1197027787566 -89.26842423836911
2002-12-31 00:00:00 948.974691119045 -16.817391798277765
2003-01-31 00:00:00 869.5098262131214 -19.66122568556662
2003-02-28 00:00:00 783.9531303942204 7.417480937313633
2003-03-31 00:00:00 743.9320541918278 93.60401913724314
2003-04-30 00:00:00 854.4385236799717 122.40532228119156
2003-05-31 00:00:00 1096.828584432602 9.834635342705987
2003-06-30 00:00:00 1797.3654448539019 180.79382312816648
2003-07-31 00:00:00 1101.0571412146091 55.169651322479325
2003-08-31 00:00:00 1989.91977

In [453]:
sort_dict_12_1 = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict_12_1, returns_df)

2001-01-31 00:00:00 1223.1137725636363 -152.65244672820612
2001-02-28 00:00:00 1469.8715858012438 -200.7768247942754
2001-03-31 00:00:00 1093.6646545231342 173.82361533419402
2001-04-30 00:00:00 1452.1372288949788 -9.5465452955242
2001-05-31 00:00:00 1206.09619198367 -34.88552407997042
2001-06-30 00:00:00 1711.3971586227417 -87.05895715952308
2001-07-31 00:00:00 1558.3936087749898 -274.51651133320837
2001-08-31 00:00:00 2213.376840427518 -440.10859508418685
2001-09-30 00:00:00 1769.1554426774383 343.1640433254644
2001-10-31 00:00:00 1420.9859773516655 99.53020332470949
2001-11-30 00:00:00 1751.6219920516014 57.049694264727876
2001-12-31 00:00:00 1859.1153335869312 -129.47160955013587
2002-01-31 00:00:00 2040.803926318884 -43.79797692711899
2002-02-28 00:00:00 1938.1204538047314 224.4776367944292
2002-03-31 00:00:00 1809.275286257267 -47.250795498868364
2002-04-30 00:00:00 3005.2582554221153 -93.34280353527126
2002-05-31 00:00:00 1210.774652659893 -128.839918256969
2002-06-30 00:00:00 2

In [459]:
sort_dict_inversion = total_return(snp_stocks_data, top_p_momentum_stocks_dict, returns_df)

2001-01-31 00:00:00 3400.226116299629 -194.94275635243017
2001-02-28 00:00:00 1303.9114480614662 -19.038545085079694
2001-03-31 00:00:00 1368.5016551017761 74.7535598389927
2001-04-30 00:00:00 1274.4073005914688 30.772693670279715
2001-05-31 00:00:00 1428.8373659849167 -26.657066631216786
2001-06-30 00:00:00 1492.7015739679337 32.27948208019485
2001-07-31 00:00:00 1398.883117198944 -53.22864415073666
2001-08-31 00:00:00 1458.383114695549 -85.39042230273748
2001-09-30 00:00:00 1457.828871667385 54.15263960360915
2001-10-31 00:00:00 1267.5143324583769 99.18894977307663
2001-11-30 00:00:00 1125.0823616981506 81.84229611366072
2001-12-31 00:00:00 1401.5908813029528 28.91127937088816
2002-01-31 00:00:00 1537.571620836854 81.99589743970897
2002-02-28 00:00:00 1539.39945936203 69.54789517759325
2002-03-31 00:00:00 1905.7627198696136 77.30259239428297
2002-04-30 00:00:00 1772.4056630581617 -36.26576679748336
2002-05-31 00:00:00 1930.2092497311532 -83.08668293077207
2002-06-30 00:00:00 1917.578

In [681]:
sort_long_dict = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict, bottom_p_momentum_stocks_dict, returns_df)

In [682]:
sort_long_dict_12_1={}
for i in sort_dict_12_1.keys():
    sort_long_dict_12_1[i] = long_dict_12_1[i] -sort_dict_12_1[i]

In [462]:
sort_long_dict_inversion={}
for i in sort_dict_inversion.keys():
    sort_long_dict_inversion[i] = long_dict_inversion[i] -sort_dict_inversion[i]

In [463]:
sort_long_dict

{Timestamp('2001-01-31 00:00:00'): 6.508475923141118,
 Timestamp('2001-02-28 00:00:00'): 11.645055235957054,
 Timestamp('2001-03-31 00:00:00'): -15.211091949787548,
 Timestamp('2001-04-30 00:00:00'): 1.7046923342779743,
 Timestamp('2001-05-31 00:00:00'): 0.6809659185431967,
 Timestamp('2001-06-30 00:00:00'): 8.013059603908516,
 Timestamp('2001-07-31 00:00:00'): 12.644518426822966,
 Timestamp('2001-08-31 00:00:00'): 13.883684544883037,
 Timestamp('2001-09-30 00:00:00'): -17.94425918926394,
 Timestamp('2001-10-31 00:00:00'): -1.0506825937679238,
 Timestamp('2001-11-30 00:00:00'): 3.6457560657541146,
 Timestamp('2001-12-31 00:00:00'): 9.152078522224324,
 Timestamp('2002-01-31 00:00:00'): 8.067560899968306,
 Timestamp('2002-02-28 00:00:00'): -8.199312323428321,
 Timestamp('2002-03-31 00:00:00'): 7.2612105362481,
 Timestamp('2002-04-30 00:00:00'): 1.381431099609221,
 Timestamp('2002-05-31 00:00:00'): 7.957928432964812,
 Timestamp('2002-06-30 00:00:00'): -3.278755997598954,
 Timestamp('2002-

In [616]:
m = 3

# Call the function
total_returns_dict_ma = total_return_with_moving_average(top_p_momentum_stocks_dict, returns_df, m, snp_stocks_data)


c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:255: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [625]:

# Apply the trading strategy
trade_returns_seasonality = seasonality_strategy(returns_df, top_p_momentum_stocks_dict)


In [ ]:

x = returns_df.loc[['GR', 'NVR', 'ODP', 'TWX', 'BKNG', 'IBM'],'2000-06-30'].abs()
x[x<=0].empty

False

# Benchmark

The S&P500 index is used as a benchmark.

In [403]:
data = benchmark_data('US', start_date)

[*********************100%%**********************]  1 of 1 completed


In [404]:
gains = benchmark_returns(data)
gains.index = pd.to_datetime(gains.index, format='%Y-%m-%d')

In [405]:
# Define a function to calculate 12-month momentum for S&P 500
def calculate_12_month_momentum(data):
    return data.pct_change(12).dropna()

benchmark = data.loc[:'2024-05-31 00:00:00']
sp500_momentum = calculate_12_month_momentum(benchmark)




# Function to compare stock momentum with S&P 500 and calculate total returns
def compare_and_calculate_returns(top_p_momentum_stocks_dict, momentum_df, returns_df, sp500_momentum, sp500_returns, max_val=50):
    # Initialize a dictionary to store the adjusted momentum returns
    adjusted_returns_dict = {}

    for timestamp, stocks in top_p_momentum_stocks_dict.items():
        stock_returns = []
        for stock in stocks:            
            if sp500_momentum.loc[timestamp] > momentum_df.loc[stock, timestamp]:
                stock_returns.append(sp500_returns.loc[timestamp])
            elif abs(returns_df.loc[stock, timestamp]) > max_val:
                stock_returns.append(0)
            else:
                stock_returns.append(returns_df.loc[stock, timestamp])
        
        total_return = sum(stock_returns)/len(stock_returns)

        adjusted_returns_dict[timestamp] = total_return

    # Convert the adjusted returns dictionary to a DataFrame
    adjusted_returns_series = pd.Series(adjusted_returns_dict)
    adjusted_returns_series.index = pd.to_datetime(adjusted_returns_series.index)

    return adjusted_returns_series

# Example usage
adjusted_returns_series = compare_and_calculate_returns(top_p_momentum_stocks_dict, momentum, returns_df, sp500_momentum, gains)


KeyError: Timestamp('2001-03-31 00:00:00')

In [406]:

results_df = calculate_yearly_stats(sort_long_dict, gains)


c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:353: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:365: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



# Plots

## 12-months momentum and 1-month return

### Going long only for the top 10% stocks

In [633]:
# Convert the total_returns_dict to a DataFrame
total_returns_df = pd.DataFrame.from_dict(long_dict, orient='index', columns=['Total Returns'])
total_returns_df.sort_index(inplace=True)


In [634]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df.index,
        y=total_returns_df['Total Returns'],
        name='Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [635]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns = (1 + total_returns_df['Total Returns'] / 100).cumprod() - 1


In [636]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns.index,
        y=cumulative_momentum_returns,
        mode='lines',
        name='Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Momentum vs Benchmark from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [577]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_12_1, gains)

yearly_stats_df.T.round(2)

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3912367007.py:5: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3912367007.py:22: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,11.41,-3.98,28.47,16.83,25.01,2.22,-4.68,-48.02,18.33,-21.44,...,-20.40,-12.49,17.96,-45.02,7.83,18.76,4.88,9.27,23.73,12.64
Benchmark Returns,-7.41,-23.37,26.38,8.99,3.00,13.62,3.53,-38.49,23.45,12.78,...,-0.73,9.54,19.42,-6.24,28.88,16.26,26.89,-19.44,24.23,5.57
Strategy Sharpe Ratio,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,...,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09,0.09
Benchmark Sharpe Ratio,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,...,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44


### Going long on top 10% and short on bottom 10%

In [687]:
# Convert the total_returns_dict to a DataFrame
total_returns_df = pd.DataFrame.from_dict(sort_long_dict, orient='index', columns=['Total Returns'])
total_returns_df.sort_index(inplace=True)


In [688]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df.index,
        y=total_returns_df['Total Returns'],
        name='Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [689]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns = (1 + total_returns_df['Total Returns'] / 100).cumprod() - 1


In [690]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns.index,
        y=cumulative_momentum_returns,
        mode='lines',
        name='Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Momentum vs Benchmark from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [691]:
results_df = calculate_yearly_stats(sort_long_dict, gains)
results_df.T

2001-12-31    10.372867
2002-12-31     6.141678
2003-12-31    -9.541986
2004-12-31     1.309659
2005-12-31    -3.292182
2006-12-31    -5.061148
2007-12-31     7.762564
2008-12-31    14.121652
2009-12-31   -16.824022
2010-12-31   -10.054501
2011-12-31    22.336565
2012-12-31    -5.843011
2013-12-31    -2.376478
2014-12-31     0.554118
2015-12-31    13.120981
2016-12-31     1.396892
2017-12-31    -7.711647
2018-12-31     3.811510
2019-12-31   -12.815382
2020-12-31   -21.008880
2021-12-31     4.009954
2022-12-31    -6.389244
2023-12-31     4.545927
2024-12-31    -0.347827
Freq: YE-DEC, dtype: float64


Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,10.372867,6.141678,-9.541986,1.309659,-3.292182,-5.061148,7.762564,14.121652,-16.824022,-10.054501,...,13.120981,1.396892,-7.711647,3.811510,-12.815382,-21.008880,4.009954,-6.389244,4.545927,-0.347827
Benchmark Returns,-7.408422,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501,5.573781
Strategy Sharpe Ratio,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,...,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229,-0.048229


### 12-1 momentum strategy

In [475]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_12_1 = pd.DataFrame.from_dict(long_dict_12_1, orient='index', columns=['Total Returns'])
total_returns_df_12_1.sort_index(inplace=True)


In [476]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_12_1.index,
        y=total_returns_df_12_1['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='12-1 Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [477]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains_12_1 = gains.reindex(total_returns_df_12_1.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_12_1 = (1 + aligned_benchmark_gains_12_1 / 100).cumprod() - 1
cumulative_momentum_returns_12_1 = (1 + total_returns_df_12_1['Total Returns'] / 100).cumprod() - 1


In [478]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_12_1.index,
        y=cumulative_benchmark_returns_12_1,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_12_1.index,
        y=cumulative_momentum_returns_12_1,
        mode='lines',
        name='12-1 Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: 12-1 Momentum vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [479]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_12_1, gains)

yearly_stats_df.T

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:353: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:365: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,11.411549,-3.979183,28.471879,16.834352,25.008793,2.220339,-4.679015,-48.021921,18.333063,-21.437321,...,-20.400222,-12.492781,17.964001,-45.023439,7.832269,18.759158,4.882551,9.270589,23.733841,12.638473
Benchmark Returns,-7.408422,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501,5.573781
Strategy Sharpe Ratio,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,...,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005,0.085005


### Buying only if the current price is above the 6-month MA

In [617]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_ma = pd.DataFrame.from_dict(total_returns_dict_ma, orient='index', columns=['Total Returns'])
total_returns_df_ma.sort_index(inplace=True)


In [618]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_ma.index,
        y=total_returns_df_ma['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [619]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains_ma = gains.reindex(total_returns_df_ma.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_ma = (1 + aligned_benchmark_gains_ma / 100).cumprod() - 1
cumulative_momentum_returns_ma = (1 + total_returns_df_ma['Total Returns'] / 100).cumprod() - 1


In [620]:
# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_ma.index,
        y=cumulative_benchmark_returns_ma,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_ma.index,
        y=cumulative_momentum_returns_ma,
        mode='lines',
        name='12-1 Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: 12-1 Momentum vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [621]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(total_returns_dict_ma, gains)

yearly_stats_df.T

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-11.645721,66.964543,24.808936,48.972371,12.061033,14.108053,-35.234865,31.784566,37.539417,24.276338,...,6.719127,-5.021179,-0.228163,10.046813,-5.700922,20.856680,86.672085,26.062708,4.968316,22.713155
Benchmark Returns,-21.809476,27.099534,9.254411,3.286917,13.787249,3.974884,-37.084043,26.304949,14.705192,1.152176,...,11.722754,0.113290,10.052217,19.500012,-5.194602,29.852081,19.867939,27.586169,-17.435621,25.423339
Strategy Sharpe Ratio,-0.084672,0.257905,0.119456,0.177474,0.067255,0.063882,-0.131231,0.104612,0.128826,0.061821,...,0.039877,-0.028213,-0.002064,0.087455,-0.025451,0.166047,0.162337,0.079477,0.015568,0.112277
Benchmark Sharpe Ratio,-0.098475,0.177281,0.102121,0.034533,0.189946,0.033586,-0.180453,0.088091,0.059559,0.005979,...,0.114446,0.000691,0.078450,0.384891,-0.028997,0.169809,0.058686,0.185516,-0.068954,0.129578


#### Buy stocks if momentum > S&P500 otherwise buy S&P500

## Inversion Strategy

#### long the bottom 10%

In [524]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_inversion = pd.DataFrame.from_dict(long_dict_inversion, orient='index', columns=['Total Returns'])
total_returns_df_inversion.sort_index(inplace=True)


In [525]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_inversion.index,
        y=total_returns_df_inversion['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Inversion Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [526]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df_inversion.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_inversions = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns_inversions = (1 + total_returns_df_inversion['Total Returns'] / 100).cumprod() - 1


In [527]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_ma.index,
        y=cumulative_benchmark_returns_ma,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_inversions.index,
        y=cumulative_momentum_returns_inversions,
        mode='lines',
        name='12-1 Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: 12-1 Momentum vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [ ]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_inversion, gains)

yearly_stats_df.T

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:5: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:17: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,-8.339661,-28.621838,64.139000,13.743506,23.023560,28.071439,-15.053489,-48.238574,80.887441,39.900061,...,-17.730483,26.741933,10.179376,-1.730873,10.380575,29.251004,31.383599,-1.820578,5.930976,2.532949
Benchmark Returns,-15.953767,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501,5.573781
Strategy Sharpe Ratio,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,...,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366
Benchmark Sharpe Ratio,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,...,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469


#### Long on bottom 10% and short on top 10%

In [665]:
# Convert the total_returns_dict to a DataFrame
s_l_total_returns_df_inversion = pd.DataFrame.from_dict(sort_long_dict_inversion, orient='index', columns=['Total Returns'])
s_l_total_returns_df_inversion.sort_index(inplace=True)


In [666]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=s_l_total_returns_df_inversion.index,
        y=s_l_total_returns_df_inversion['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Inversion Return(sort and long) vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [667]:
# Ensure both DataFrames have the same index for correct comparison
s_l_aligned_benchmark_gains = gains.reindex(s_l_total_returns_df_inversion.index).fillna(0)

# Calculate cumulative returns
s_l_cumulative_benchmark_returns_inversions = (1 + s_l_aligned_benchmark_gains / 100).cumprod() - 1
s_l_cumulative_momentum_returns_inversions = (1 + s_l_total_returns_df_inversion['Total Returns'] / 100).cumprod() - 1


In [668]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_inversions.index,
        y=cumulative_benchmark_returns_inversions,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=s_l_cumulative_momentum_returns_inversions.index,
        y=s_l_cumulative_momentum_returns_inversions,
        mode='lines',
        name='Momentum inversion Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Inversion vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [ ]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(sort_long_dict_inversion, gains)

yearly_stats_df.T

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:5: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:17: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,-9.613155,-8.778563,19.216087,4.995825,-8.615698,9.805042,-23.200976,-6.136139,37.104537,7.822143,...,-14.065914,14.457479,-19.106177,-1.846799,-5.161210,6.833471,21.271645,-11.482898,-12.564045,-5.102908
Benchmark Returns,-15.953767,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501,5.573781
Strategy Sharpe Ratio,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,...,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547
Benchmark Sharpe Ratio,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,...,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469


### Seasonality

#### Buy from November to May

In [626]:
# Ensure both DataFrames have the same index for correct comparison
strategy_gains = trade_returns_seasonality.reindex(gains.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + gains / 100).cumprod() - 1
seasonality_cumulative_momentum_returns = (1 + strategy_gains['Trade Returns'] / 100).cumprod() - 1


In [627]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=seasonality_cumulative_momentum_returns.index,
        y=seasonality_cumulative_momentum_returns,
        mode='lines',
        name='Momentum inversion Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Inversion vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [628]:
# Calculate the yearly stats
returns_dict = strategy_gains['Trade Returns'].to_dict()
yearly_stats_df = calculate_yearly_stats(returns_dict, gains)

yearly_stats_df.T

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-0.651321,17.707897,5.426718,25.619930,6.431499,-1.451825,10.243661,3.226519,8.934175,14.422276,...,11.383619,0.080494,13.090768,9.839418,4.011624,12.345799,6.217428,6.595308,2.771134,6.419306
Benchmark Returns,-21.809476,27.099534,9.254411,3.286917,13.787249,3.974884,-37.084043,26.304949,14.705192,1.152176,...,11.722754,0.113290,10.052217,19.500012,-5.194602,29.852081,19.867939,27.586169,-17.435621,25.423339
Strategy Sharpe Ratio,-0.012233,0.186803,0.058056,0.112544,0.047395,-0.009473,0.053902,0.013744,0.042535,0.167238,...,0.090661,0.000776,0.130635,0.087911,0.052355,0.101552,0.025560,0.055783,0.011035,0.058362
Benchmark Sharpe Ratio,-0.098475,0.177281,0.102121,0.034533,0.189946,0.033586,-0.180453,0.088091,0.059559,0.005979,...,0.114446,0.000691,0.078450,0.384891,-0.028997,0.169809,0.058686,0.185516,-0.068954,0.129578


______________________________________________________________________________________________________________________________________________________

In [607]:

def calculate_sharpe_ratio(monthly_returns, risk_free_rate=0):
    # Calculate the average monthly return
    average_monthly_return = monthly_returns.mean()
    
    # Calculate the standard deviation of the monthly returns
    monthly_std_dev = monthly_returns.std()
    
    # Calculate the annualized return
    annualized_return = (1 + average_monthly_return / 100) ** (12) - 1
    
    # Calculate the annualized standard deviation
    annualized_std_dev = monthly_std_dev * np.sqrt(12)
    
    # Calculate the Sharpe ratio
    sharpe_ratio = (average_monthly_return - risk_free_rate) / annualized_std_dev
    
    return sharpe_ratio, annualized_return * 100  # Return annualized return as a percentage

def calculate_yearly_stats(strategy_dict, benchmark_returns, risk_free_rate=0):
    # Convert the strategy dictionary to a DataFrame
    strategy_returns_df = pd.Series(strategy_dict)
    strategy_returns_df.index = pd.to_datetime(strategy_returns_df.index)
    strategy_returns_df = strategy_returns_df.resample('ME').last()  # Ensure monthly frequency
    
    # Convert the benchmark returns to a DataFrame
    benchmark_returns.index = pd.to_datetime(benchmark_returns.index)
    benchmark_returns = benchmark_returns.resample('ME').last()  # Ensure monthly frequency
    
    # Initialize lists to store yearly results
    years = []
    strategy_returns = []
    benchmark_returns_list = []
    strategy_sharpe_ratios = []
    benchmark_sharpe_ratios = []

    # Calculate stats for each year
    for year in strategy_returns_df.index.year.unique():
        # Filter the returns for the current year
        strategy_yearly_returns = strategy_returns_df[strategy_returns_df.index.year == year]
        benchmark_yearly_returns = benchmark_returns[benchmark_returns.index.year == year]

        # Ensure we have 12 months of data
        if len(strategy_yearly_returns) == 12 and len(benchmark_yearly_returns) == 12:
        
            # Calculate Sharpe ratio
            strategy_sharpe_ratio, strategy_annual_return = calculate_sharpe_ratio(strategy_yearly_returns, risk_free_rate)
            benchmark_sharpe_ratio, benchmark_annual_return = calculate_sharpe_ratio(benchmark_yearly_returns, risk_free_rate)

            # Append results
            years.append(year)
            strategy_returns.append(strategy_annual_return)
            benchmark_returns_list.append(benchmark_annual_return)
            strategy_sharpe_ratios.append(strategy_sharpe_ratio)
            benchmark_sharpe_ratios.append(benchmark_sharpe_ratio)

    # Create a DataFrame to store the results
    results_df = pd.DataFrame({
        'Year': years,
        'Strategy Returns': strategy_returns,
        'Benchmark Returns': benchmark_returns_list,
        'Strategy Sharpe Ratio': strategy_sharpe_ratios,
        'Benchmark Sharpe Ratio': benchmark_sharpe_ratios
    })

    results_df.set_index('Year', inplace=True)    
    
    return results_df

In [608]:
yearly_stats_df = calculate_yearly_stats(long_dict_12_1, gains)

yearly_stats_df.T.round(2)

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-2.70,29.55,18.24,27.61,3.77,-2.96,-43.65,20.09,-19.05,-14.85,...,-6.32,-17.31,-10.17,19.94,-43.16,8.45,23.31,9.64,9.89,25.91
Benchmark Returns,-21.81,27.10,9.25,3.29,13.79,3.97,-37.08,26.30,14.71,1.15,...,11.72,0.11,10.05,19.50,-5.19,29.85,19.87,27.59,-17.44,25.42
Strategy Sharpe Ratio,-0.01,0.16,0.09,0.10,0.02,-0.01,-0.12,0.09,-0.07,-0.04,...,-0.02,-0.06,-0.04,0.08,-0.18,0.06,0.06,0.02,0.07,0.10
Benchmark Sharpe Ratio,-0.10,0.18,0.10,0.03,0.19,0.03,-0.18,0.09,0.06,0.01,...,0.11,0.00,0.08,0.38,-0.03,0.17,0.06,0.19,-0.07,0.13


In [611]:
results_df = yearly_stats_df
# Plot the Sharpe ratios
fig = go.Figure()
    
    # Add strategy Sharpe ratios bar trace
fig.add_trace(
    go.Bar(
        x=results_df.index,
        y=results_df['Strategy Sharpe Ratio'],
        name='Strategy Sharpe Ratio'
    )
)
    
    # Add benchmark Sharpe ratios bar trace
fig.add_trace(
    go.Bar(
        x=results_df.index,
        y=results_df['Benchmark Sharpe Ratio'],
        name='Benchmark Sharpe Ratio'
    )
)
    
    # Update the layout
fig.update_layout(
    title='Yearly Strategy vs Benchmark Sharpe Ratios',
    xaxis_title='Year',
    yaxis_title='Sharpe Ratio',
    barmode='group'
)
    
fig.show()

In [593]:
long_dict

{Timestamp('2001-01-31 00:00:00'): -5.360236191011982,
 Timestamp('2001-02-28 00:00:00'): -1.4601102792206038,
 Timestamp('2001-03-31 00:00:00'): 5.462438394598306,
 Timestamp('2001-04-30 00:00:00'): 3.150151662808117,
 Timestamp('2001-05-31 00:00:00'): -1.865647362381352,
 Timestamp('2001-06-30 00:00:00'): 2.162487307787101,
 Timestamp('2001-07-31 00:00:00'): -3.805081603766806,
 Timestamp('2001-08-31 00:00:00'): -5.855143373664438,
 Timestamp('2001-09-30 00:00:00'): 3.765097253080987,
 Timestamp('2001-10-31 00:00:00'): 7.825469679754791,
 Timestamp('2001-11-30 00:00:00'): 7.274338208460712,
 Timestamp('2001-12-31 00:00:00'): 2.062747393448474,
 Timestamp('2002-01-31 00:00:00'): 5.332818083302102,
 Timestamp('2002-02-28 00:00:00'): 4.517858880268532,
 Timestamp('2002-03-31 00:00:00'): 4.056254831114115,
 Timestamp('2002-04-30 00:00:00'): -2.0461324150200086,
 Timestamp('2002-05-31 00:00:00'): -4.304542781687773,
 Timestamp('2002-06-30 00:00:00'): -8.179458790841373,
 Timestamp('2002-0

In [664]:
sort_long_dict = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict, bottom_p_momentum_stocks_dict, returns_df)

In [646]:
total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

{Timestamp('2001-01-31 00:00:00'): -11.510414729898818,
 Timestamp('2001-02-28 00:00:00'): -0.398165117415966,
 Timestamp('2001-03-31 00:00:00'): 12.572787247712508,
 Timestamp('2001-04-30 00:00:00'): 0.25650192165149954,
 Timestamp('2001-05-31 00:00:00'): -1.923829588527738,
 Timestamp('2001-06-30 00:00:00'): -5.850572296121416,
 Timestamp('2001-07-31 00:00:00'): -16.44960003058977,
 Timestamp('2001-08-31 00:00:00'): -4.4144535120076265,
 Timestamp('2001-09-30 00:00:00'): 0.6528550016546848,
 Timestamp('2001-10-31 00:00:00'): 7.183458393504679,
 Timestamp('2001-11-30 00:00:00'): 4.842498070753337,
 Timestamp('2001-12-31 00:00:00'): -7.08933112877585,
 Timestamp('2002-01-31 00:00:00'): -0.5798539494056716,
 Timestamp('2002-02-28 00:00:00'): 11.512150800954087,
 Timestamp('2002-03-31 00:00:00'): -0.4369759692068638,
 Timestamp('2002-04-30 00:00:00'): -3.4275635146292296,
 Timestamp('2002-05-31 00:00:00'): -9.421832958482009,
 Timestamp('2002-06-30 00:00:00'): -5.335706688418725,
 Timest

In [520]:
mont_r = d.resample('ME').last()

In [513]:
x = pd.DataFrame(mont_r.pct_change(-1).loc['2001-01-31':'2024-04-30'])

In [523]:
x

,Adj Close
Date,
2001-01-31,-0.065673
2001-02-28,0.106539
2001-03-31,0.083221
2001-04-30,-0.095204
2001-05-31,0.004577
...,...
2023-12-31,-0.031247
2024-01-31,-0.033297
2024-02-29,-0.044946


In [514]:
(1 + x['Adj Close']).cumprod()-1

Date
2001-01-31   -0.065673
2001-02-28    0.033870
2001-03-31    0.119909
2001-04-30    0.013289
2001-05-31    0.017927
                ...   
2023-12-31   -0.729429
2024-01-31   -0.738439
2024-02-29   -0.750195
2024-03-31   -0.755277
2024-04-30   -0.744287
Freq: ME, Name: Adj Close, Length: 280, dtype: float64

In [521]:
mont_r

Date
2000-01-31    1394.459961
2000-02-29    1366.420044
2000-03-31    1498.579956
2000-04-30    1452.430054
2000-05-31    1420.599976
                 ...     
2024-02-29    5096.270020
2024-03-31    5254.350098
2024-04-30    5035.689941
2024-05-31    5277.509766
2024-06-30    5283.399902
Freq: ME, Name: Adj Close, Length: 294, dtype: float64

In [522]:
mont_r.shift(-1)

Date
2000-01-31    1366.420044
2000-02-29    1498.579956
2000-03-31    1452.430054
2000-04-30    1420.599976
2000-05-31    1454.599976
                 ...     
2024-02-29    5254.350098
2024-03-31    5035.689941
2024-04-30    5277.509766
2024-05-31    5283.399902
2024-06-30            NaN
Freq: ME, Name: Adj Close, Length: 294, dtype: float64